## Дипломный проект на курсе Data Science школа SkillFactory
# «Модель прогнозирования стоимости жилья для агентства недвижимости»

**Цель:** разработать модель, которая позволила бы агентству недвижимости обойти конкурентов по скорости и качеству совершения сделок.

**Поставленные задачи:**
1. Провести разведывательный анализ и очистку исходных данных. Обратите внимание, что данные в таблице реальные: в результате во многих признаках присутствуют дублирующиеся категории, ошибки ввода, жаргонные сокращения и т .д. Вам предстоит отыскать закономерности, самостоятельно расшифровать все сокращения, найти синонимы в данных, обработать пропуски и удалить выбросы. 
2. Выделить наиболее значимые факторы, влияющие на стоимость недвижимости.
3. Построить модель для прогнозирования стоимости недвижимости.
4. Разработать небольшой веб-сервис, на вход которому поступают данные о некоторой выставленной на продажу недвижимости, а сервис прогнозирует его стоимость.

**Описание данных:**<br>
<br>
➔ 'status' — статус продажи;<br>
➔ 'private pool' и 'PrivatePool' — наличие собственного бассейна;<br>
➔ 'propertyType' — тип объекта недвижимости;<br>
➔ 'street' — адрес объекта;<br>
➔ 'baths' — количество ванных комнат;<br>
➔ 'homeFacts' — сведения о строительстве объекта (содержит несколько
типов сведений, влияющих на оценку объекта);<br>
➔ 'fireplace' — наличие камина;<br>
➔ 'city' — город;<br>
➔ 'schools' — сведения о школах в районе;<br>
➔ 'sqft' — площадь в футах;<br>
➔ 'zipcode' — почтовый индекс;<br>
➔ 'beds' — количество спален;<br>
➔ 'state' — штат;<br>
➔ 'stories' — количество этажей;<br>
➔ 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система
мультилистинга);<br><br>

➔ **'target'** — цена объекта недвижимости (целевой признак, который
необходимо спрогнозировать).

In [207]:
# подключаем необходимые библиотеки
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re

In [208]:
# зафиусируем RANDOM_SEED
RANDOM_SEED = 45

In [209]:
# зафиксируем версию пакетов
!pip freeze > requirements.txt

## 1. Разведывательный анализ и очистка исходных данных.

In [210]:
df = pd.read_csv(r'D:\Diplom\data.csv')
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [211]:
# информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [212]:
count_rows = df.shape[0]
print(f'Количество строк в базе данных: {count_rows}')

Количество строк в базе данных: 377185


В базе данных 377185 строк и 17 признаков + столбец таргета. Все данные типа object. Практически во всех столбцах имеются пустые значения.<br>
Первоначально удалим полные дубликаты строк если они имеются.

### Поиск и удаление дубликатов

In [213]:
#удаляем дубликаты строк
df = df.drop_duplicates(ignore_index=True)

# посчитаем количество удаленных дубликатов
duplicate_rows = count_rows - df.shape[0]
print(f'Удалено дубликатов: {duplicate_rows}')

Удалено дубликатов: 50


### Исследование пустых значений

In [214]:
df.isnull().mean() * 100

status          10.584274
private pool    98.891378
propertyType     9.209699
street           0.000530
baths           28.188315
homeFacts        0.000000
fireplace       72.659393
city             0.009015
schools          0.000000
sqft            10.752118
zipcode          0.000000
beds            24.196640
state            0.000000
stories         39.952272
mls-id          93.386453
PrivatePool     89.311520
MlsId           17.730786
target           0.657589
dtype: float64

Столбцы mls-id и MlsId это внутренние риэлторские метки, не влияющими на стоимости недвижимости, поэтому данные столбцы можно удалить.

In [215]:
# Удаляем столбцы MlsId и mls-id
df = df.drop(['MlsId', 'mls-id'], axis=1)

### Данные в столбцах private pool и PrivatePool, stories и fireplace

Больше всего пропусков в столбцах: PrivatePool и private pool, > 89%. Т.к. смысловая нагрузка у столбцов одинаковая объединим их, при этом будем считать пропуски в них как отсутствие бассейна.

In [216]:
# проверим какие данные находятся в столбцах
print(df['private pool'].unique())
print(df['PrivatePool'].unique())

# объеденим данные столбцов private pool и PrivatePool и поместим их в новый столбец private_pool_all, меняем данные на булевые переменные
df['private_pool_all'] = df['private pool'].fillna('') + df['PrivatePool'].fillna('')
df['private_pool_all'] = df['private_pool_all'].replace(['yes', 'Yes'], True)
df['private_pool_all'] = df['private_pool_all'].replace('', False)
# проеверим новый столбец на уникальность данных
df['private_pool_all'].unique()
# удаляем столбцы private pool и PrivatePool 
df = df.drop(['private pool', 'PrivatePool'], axis=1)

[nan 'Yes']
[nan 'yes' 'Yes']


Столбец fireplace заполнен данными чуть более чем на четверть. Пустота столбца составляет 73%. Проверим содержание столбца.

In [217]:
df['fireplace'].value_counts()

fireplace
yes                                                                     50353
Yes                                                                     20856
1                                                                       14544
2                                                                        2432
Not Applicable                                                           1993
                                                                        ...  
Free-standing, Insert, Wood                                                 1
Wood Burning, Attached Fireplace Doors/Screen, Electric, Gas Starter        1
One, Living Room                                                            1
FAMILYRM, Great Room, Living Room                                           1
Ceiling Fan, SMAPL, Utility Connection, Walk-In Closets                     1
Name: count, Length: 1652, dtype: int64

Очевидно что в столбец "наличие камина" (fireplace), записано много общей информации по объекту (наличие инженерных сетей, гардеробных, лес и т.д.). При этом уникальных записей в столбце 1652 и очень велика доля пустых значений (73%). Поэтому считаем, что данныей столбец не информативен, что позволяет нам его удалить. <br>

В столбце количества этажей (stories) 40% данных отсутствует, определить её не представляется возможным, придется так же удалить данный столбец.<br>

In [218]:
df = df.drop(['stories', 'fireplace'], axis=1)

### Пропуски и обработка столбца beds.

In [219]:
#Проверим данные в столбце
df['beds'].value_counts()

beds
3 Beds         53454
4 Beds         35412
3              31405
2 Beds         26355
4              20030
               ...  
8,023 sqft         1
10,193 sqft        1
8.93 acres         1
5,510 sqft         1
8,479 sqft         1
Name: count, Length: 1184, dtype: int64

Анализируя данные в столбце становится видно, что они имеют данные из других столбцов типа sqft. Постараемся их сохронить и при возможности учесть их при обработке этих столбцов. Для этого запишем их в другой столбец.

In [220]:
df['sqft_from_beds'] = df['beds'].str.extract(r'(\d+,\d+)\s+sqft', expand=False)
df['acres_from_beds'] = df['beds'].str.extract(r'([\d.]+)\s+acres', expand=False)

In [221]:
# Удаляем перенесенные данные
df['beds'] = df['beds'].str.replace(r'(\d+,\d+)\s+sqft', '', regex=True)
df['beds'] = df['beds'].str.replace(r'([\d.]+)\s+acres', '', regex=True)

# Преобразуем некоторые значения в общий вид
df['beds'] = df['beds'].replace(['1 Bath, 2 Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room', '1 Bath, 2 Bedrooms, Living Room, Range/Oven, Refrigerator', '1 Bath, 2 Bedrooms', '1 Bath, 2 Bedrooms, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator'], 2)
df['beds'] = df['beds'].replace(['1 Bath, 3 or More Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator', '3 or More Bedrooms, Dining Room, Living Room, Range/Oven, Refrigerator', '3 or More Bedrooms', '2 Baths, 3 or More Bedrooms'], 3)
df['beds'] = df['beds'].replace(["Based on Redfin's St Johns data, we estimate the home's value is $360,731, which is 2.2% less than its current list price.", "Based on Redfin's Raleigh data, we estimate the home's value is $708,248, which is 1.2% more than its current list price.", '-- bd', '-- sqft', '4 sqft', '60 sqft', '1 acre', '840 sqft', '540 sqft', '100 sqft', '871 sqft', '640 sqft', '831 sqft', '448 sqft', '248 sqft', '# Bedrooms 1st Floor'], 0)

# Выделим из имеющихся данных числа и приведем их в "нормальный" вид
df['beds_a1'] = df['beds'].str.extract(r'(\d+)\s*(?:Beds|bd)', expand=False)
df['beds_a2'] = df['beds'].str.extract(r'(\d+\.\d+|\d+)')

# Создаем столбец в котором данные из beds_a1 b beds_a2  будут объеденены, а пропуски заполнены 0
df['beds_all'] = df['beds_a1'].fillna(0)
df['beds_all'] = df['beds_a2'].fillna(0)

# Проверим данные на пустые строки
beds_nulldata = df['beds_all'].isnull().mean()*100
print(f'Пустых строк в столбце beds_all: {beds_nulldata} %')

# Изменим тип данных для удобной обработки их в дальнейшем
df['beds_all'] = df['beds_all'].astype(float)

# Удалим столбцы beds, beds_a1 и beds_a2 (дабы не захламлять данные)
df = df.drop(['beds_a1', 'beds_a2', 'beds'], axis=1)

Пустых строк в столбце beds_all: 0.0 %


### Пропуски и обработка столбца sqft.

In [222]:
# посмотрим на данные в столбце
df['sqft'].value_counts()

sqft
0                                          11854
1,200 sqft                                   839
1,000 sqft                                   654
1,100 sqft                                   573
1,800 sqft                                   563
                                           ...  
9,914                                          1
Total interior livable area: 3,055 sqft        1
5,177                                          1
11620                                          1
Total interior livable area: 4,615 sqft        1
Name: count, Length: 25405, dtype: int64

In [223]:
# Проверим содержание столбца на пустые строки
sqft_isnulldata = df['sqft'].isnull().mean()*100
print(f'Процент пустых строк в столбце: {sqft_isnulldata:.2f} %')

Процент пустых строк в столбце: 10.75 %


In [224]:
# Уберем лишние значения и оставим только целочисленные
df['sqft'] = df['sqft'].str.replace(',', '').str.extract('(\d+)')

# Проверим соделжание столбца на пустые строки
sqft_dataisnull = df['sqft'].isnull().sum()
print(f'Количество пустых строк sqft: {sqft_dataisnull}')

Количество пустых строк sqft: 41370


Пытаемся дополнить данные в столбце sqft из столбцов sqft_from_beds и acres_from_beds

In [225]:
# Удаляем лишние знаки
df['sqft_from_beds'] = df['sqft_from_beds'].replace(',', '', regex=True)

# Дополняем столбец данными из sqft_from_beds
df['sqft'] = df['sqft'].fillna(df['sqft_from_beds'])

# Проверим содержание столбца на пустые строки теперь стало
sqft_dataisnull_1 = df['sqft'].isnull().sum()
print(f'Количество пустых строк в столбце после дополнения: {sqft_dataisnull_1}')

Количество пустых строк в столбце после дополнения: 40049


In [226]:
# Посмотрим содержание столбца
df['acres_from_beds'].value_counts()

acres_from_beds
0.34    40
0.28    39
0.26    32
0.29    31
1.03    30
        ..
2.44     1
4.77     1
1.66     1
2.68     1
1.74     1
Name: count, Length: 446, dtype: int64

In [227]:
# Создадим новый столбец в котором будет записано числовое значение перевода акров в футы (1 акр = 43560 футов кв.)
df['acres_from_beds'] = df['acres_from_beds'].astype(float)
df['acres_from_beds_ft'] = df['acres_from_beds'] * 43560

# Дополняем столбец данными из acres_from_beds_ft
df['sqft'] = df['sqft'].fillna(df['acres_from_beds_ft'])

# Проверим содержание столбца на пустые строки теперь стало
sqft_dataisnull_2 = df['sqft'].isnull().sum()
print(f'Количествопустых строк в столбце после второго дополнения: {sqft_dataisnull_2}')

Количествопустых строк в столбце после второго дополнения: 38458


In [228]:
# Заполним пустые строки значением 0
df['sqft'] = df['sqft'].fillna(0)

# Изменим тип данных для удобства дальнейшей их обработки
df['sqft'] = df['sqft'].astype(int)

# Удалим отработанные столбцы sqft_from_beds, acres_from_beds и acres_from_beds_ft
df = df.drop(['sqft_from_beds', 'acres_from_beds', 'acres_from_beds_ft'], axis=1)

### Пропуски и обработка столбца baths.

In [229]:
# посмотрим на данные в столбце
df['baths'].value_counts()

baths
2 Baths       52458
3 Baths       35503
2             20452
2.0           16575
4 Baths       14763
              ...  
4.75 Baths        1
5.25 Baths        1
41.0              1
1.8 Baths         1
44.0              1
Name: count, Length: 229, dtype: int64

In [230]:
# Проверим содержание столбца на пустые строки
baths_isnulldata = df['baths'].isnull().mean()*100
print(f'Процент пустых строк в столбце: {baths_isnulldata:.2f} %')

Процент пустых строк в столбце: 28.19 %


In [231]:
# Приведем данные к единому виду, удалим из данных буквы и пробелы
df['baths'] = df['baths'].str.replace('[a-zA-Z+:]','', regex=True)
df['baths'] = df['baths'].replace(['~', '0 / 0', '. . ', '-- ', '—'], 0)
df['baths'] = df['baths'].replace(['1 / 1-0 / 1-0 / 1-0','1-0 / 1-0 / 1', '1 / 1 / 1 / 1', '1-2 ', '1,000'], 1)
df['baths'] = df['baths'].replace(['2-1 / 2-1 / 1-1 / 1-1', '2,000'], 2)
df['baths'] = df['baths'].replace(['3-1 / 2-2', '3,000'], 3)
df['baths'] = df['baths'].replace('116 / 116 / 116', 116)
df['baths'] = df['baths'].replace('7,500', 7.5)
df['baths'] = df['baths'].replace('5,000', 5)
df['baths'] = df['baths'].replace('3,500', 3.5)
df['baths'] = df['baths'].replace('2,250', 2.25)
df['baths'] = df['baths'].replace('1,250', 1.25)
df['baths'] = df['baths'].replace('2,500', 2.5)
df['baths'] = df['baths'].replace('2,750', 2.75)
df['baths'] = df['baths'].replace('4,000', 4)
df['baths'] = df['baths'].replace('1,750', 1.75)
df['baths'] = df['baths'].replace('1,500', 1.5)

# Удалим значения которые не являются числом
df['baths'] = df['baths'].str.extract(r'(\d+\.\d+|\d+)')

# Заполним пропуски 0 
df['baths'] = df['baths'].fillna(0)

# Изменим тип данных для удобства дальнейшей их обработки
df['baths'] = df['baths'].astype(float)

df['baths'].value_counts()

baths
0.0     126124
2.0     101714
3.0      64992
4.0      26042
1.0      16503
         ...  
14.5         1
25.0         1
55.0         1
76.0         1
68.0         1
Name: count, Length: 83, dtype: int64

### Пропуски и обработка столбца city.

In [232]:
# Посмотрим на данные в столбце
df['city'].value_counts()

city
Houston            24441
San Antonio        15592
Miami              15524
Jacksonville       10015
Dallas              8855
                   ...  
New Albany             1
Los Altos Hills        1
Lake worth             1
Lisle                  1
Blue Springs           1
Name: count, Length: 2026, dtype: int64

In [233]:
# найдем пустые строки в столбце city
city_isnulldata = df['city'].isnull()
print(city_isnulldata)

0         False
1         False
2         False
3         False
4         False
          ...  
377130    False
377131    False
377132    False
377133    False
377134    False
Name: city, Length: 377135, dtype: bool


При анализе данных в столбце city выявлено небольшое количество отсутствующих данных. При этом в предоставленной базе данныех имеется полная информация о почтовом индексе - из которой можно найти город.
Найдем список индексов в нашей базе данных у которых нет городов  

In [234]:
df[city_isnulldata]['zipcode'].value_counts()

zipcode
78045    6
34747    4
34474    3
33955    3
32686    2
34432    2
34481    2
32179    2
38732    1
33126    1
34744    1
34741    1
34488    1
34473    1
32668    1
33954    1
20003    1
77032    1
Name: count, dtype: int64

Список отсутствующих городов не так уж и велик. Пробуем восстановить информацию из столбца zipcode вручную. 

In [235]:
zipcode_city_list = {'20003':'Washington', '32179':'Ocklawaha', '32668':'Morriston', '32686':'Reddick', '33126':'Miami', '33954':'Port Charlotte',
                     '33955':'Punta Gorda', '34432':'Dunnellon','34473':'Ocala', '34474':'Ocala', '34481':'Ocala', '34488':'Silver Springs',
                     '34741':'KISSIMMEE', '34744':'Kissimmee', '34747':'Kissimmee	', '38732':'Cleveland', '77032':'Houston', '78045':'Laredo'}
# Перебираем значения с отсутствующими городами и сверяем с имеющимся списком 
def find_city (zipcode):
    if zipcode in zipcode_city_list:
        return zipcode_city_list[zipcode]
    else:
        return None  # если zipcode не найден, возвращаем None

df['city_add'] = df['zipcode'].apply(find_city)

# применяем функцию к столбцу 'zipcode' и записываем результаты в столбец 'city_add'
df['city_add'] = df['zipcode'].apply(find_city)

# Дополняем данные в столбце city данными из столбца city_add
df['city'].fillna(df['city_add'], inplace=True)

# Проверяем столбец на наличие пустых строк
city_isnulldata_new = df['city'].isnull().mean()*100
print(f'Процент пропущенных строк в столбце city: {city_isnulldata_new:.2f} %')

# Удаляем столбец city_add
df = df.drop(['city_add'], axis=1)

Процент пропущенных строк в столбце city: 0.00 %


### Пропуски и обработка столбца status

In [236]:
# Проверим содержание столбца
#df['status'].value_counts()
uniq_status_sum = df['status'].nunique()
print(f'Количество уникальных значений столбца status: {uniq_status_sum}')

df['status'].unique()[:20]

Количество уникальных значений столбца status: 159


array(['Active', 'for sale', nan, 'New construction', 'New', 'For sale',
       'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction',
       ' / auction', 'Under Contract', 'Under Contract   Showing',
       'Pre-foreclosure', 'Under Contract Backups', 'foreclosure',
       'Active Under Contract', 'Foreclosed', 'Option Pending',
       'Under Contract Show'], dtype=object)

In [237]:
# Проверим количество пустых строк в столбце
df['status'].isnull().mean()*100

10.584273536001696

В столбце имеется большое количестов уникальных значений. Попробуем сократить число уникальных значений путем приведения похожих данных к одному виду. Для этого создадим словарь значений объединяющий все примерно одинаковые значения и нормальное значение. 
Имеющиеся пропуски заменим на значение 'unknown'

In [238]:
values_status = {
    "For sale": ["for sale", "For sale", "New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"],
    "Under contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"],
    "For rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction", "Pre-foreclosure", "Pre-foreclosure / auction", "/ auction", "Foreclosed", "foreclosure", "Foreclosure"],
    "Due diligence": ["Due Diligence Period"],
    "Recently sold": ["recently sold"],
    "Price change": ["Price Change"],
    "Back on market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing extended": ["Listing Extended"],
    "Coming soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."],
}

def change_status(row):
    for status_new, values in values_status.items():
        if row in values:
            return status_new
    return "unknown"

Произведем замену значений в столбце 'status' в соответствии нашим словарем

In [239]:
df['status'] = df['status'].apply(change_status)

# Ещё раз проверим количество уникальныз значений в столбце
uniq_status_sum = df['status'].nunique()
print(f'Количество уникальных значений столбца status: {uniq_status_sum}')

Количество уникальных значений столбца status: 15


### Пропуски и обработка столбца propertyType

In [240]:
# Проверим содержание столбца
uniq_propertyType_sum = df['propertyType'].nunique()
print(f'Количество уникальных значений столбца propertyType: {uniq_propertyType_sum}')

df['propertyType'].unique()[:20]

Количество уникальных значений столбца propertyType: 1280


array(['Single Family Home', 'single-family home', 'lot/land',
       'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English',
       '2 Story', 'Townhouse', 'multi-family', 'Penthouse, Split-Level',
       'Multi-Family Home', 'Condo', 'condo', 'Land',
       'Condo/Townhome/Row Home/Co-Op', ' ', 'Detached, Two Story'],
      dtype=object)

In [241]:
# смотрим сколько пропусков в столбце
df['propertyType'].isnull().sum()

34733

Приведем к единой форме одинаковые по смыслу данные, остальные заменим на значение 'other'

In [242]:
# Приведем все в нижний регистр
df['propertyType'] = df['propertyType'].str.lower()

# Создадим 9 основных групп, все остальные будут unknown
df['propertyType'] = df['propertyType'].replace(['single-family home', 'single family home', 'singlefamilyresidence', '1 story',
                                                'one story traditional', '1 story traditional', 'detached, one story',
                                                'single detached', 'single wide', 'single-wide mobile with land',
                                                '1 story, contemporary', '1 story, other (see remarks)',
                                                'single detached, french', '1 story, traditional', 'single detached, traditional',
                                                'one story', 'one level unit', '1 1/2 story', 'single wide mh',
                                                '1 story,traditional', '1 story, historic/older, traditional', '1 story, split level'],
                                        'single family')
df['propertyType'] = df['propertyType'].replace(['multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex',
                                                 'detached, two story', '2 story, other (see remarks)',
                                                 'multi_level', '2-story', 'two story', 'multi-level, modern', '2 stories, traditional',
                                                 '2 stories', 'traditional', 'attached or 1/2 duplex, traditional'],
                                        'multi family')
df['propertyType'] = df['propertyType'].replace(['coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome','condominium',
                                                 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise',
                                                 '2 unit condo', 'condo/townhome, contemporary/modern, loft, traditional',
                                                 'condo/townhome, hi-rise, resort property, vacation home, contemporary/modern',
                                                 'condo/townhome, other (see remarks)', 'condo/townhome, french',
                                                 'condo/townhome, hi-rise, contemporary/modern, loft', 'condo, other (see remarks)',
                                                 'condo/townhome, craftsman, traditional'],
                                        'condo')
df['propertyType'] = df['propertyType'].replace(['townhome style', 'townhouse-interior', 'townhouse-end unit',
                                                 'townhouse, attached/row', 'townhouse, northwestern contemporary', 'attached, townhouse',
                                                 'townhouse, 2-story', 'townhouse, two story, traditional', 'townhouse, villa'],
                                        'townhouse')
df['propertyType'] = df['propertyType'].replace(['lot/land'],
                                        'land')
df['propertyType'] = df['propertyType'].replace(['mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home',
                                                 'manufactured home', 'manufactured double-wide', 'manufactured single-wide',
                                                 'mobile home 1 story', 'mobile manu - double wide','manufactured house,ranch, one story',
                                                 'manufactured house, ranch, one story, manufactured home',
                                                 'manufactured house, traditional, manufactured home', 'manufactured house, manufactured home',
                                                 '1 story, manufactured home - single wide', 'manufactured home, mobile home, ranch'],
                                        'mobile home')
df['propertyType'] = df['propertyType'].replace(['designated historical home', 'historical/conservation district', 'historic/older',
                                                 'historic vintage', 'historic', '1 story, historic/older, craftsman',
                                                 'historical/conservation district, single detached, contemporary/modern, traditional',
                                                 '1 story, historic/older, traditional, craftsman', 'historical, traditional',
                                                 '2 stories, historic/older, craftsman'],
                                        'historical')
df['propertyType'] = df['propertyType'].replace(['rancher', '1 story/ranch', '1 story, ranch', 'rancher, raised ranch', 'farms/ranches',
                                                 'hi ranch', 'ranch, one story', 'ranch, traditional', 'farm house',
                                                 'ranch, one story, duplex', 'farm house', 'ranch, transitional',
                                                 'bungalow, contemporary, ranch, traditional', '1 story, ranch, traditional, texas hill country',
                                                 '1 story, contemporary, ranch, historic/older, traditional', 'farm house, transitional',
                                                 'ranch, traditional, transitional', 'old world, ranch', 'ranch, split level', 
                                                 'ranch, spanish', 'farm/ranch house, single detached, contemporary/modern, ranch',
                                                 '2 stories, colonial, ranch', '1 story, ranch, craftsman', 'raised ranch, rancher'],
                                        'ranch')
df['propertyType'] = df['propertyType'].replace(['condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)',
                                                 'Flats', 'studio'],
                                        'apartment')
df['propertyType'] = df['propertyType'].replace('yes','other')

# Заменим пустые строки на other
df['propertyType'].fillna('other', inplace=True)

# Основные группы
type_main = ['single family','multi family', 'condo', 'townhouse', 'land', 'mobile home', 'historical', 'ranch', 'apartment', 'unknown']
# Заменим значения не входящие в группы на other
def change_type(type):
    if type in type_main:
        return type    # Значение найдено в списке, остается без изменений
    else:
        return 'other'

# Проиграем вновь написанную функцию
df['propertyType'] = df['propertyType'].apply(change_type)

# Ещё раз проверим столбец на уникальные значения и пропуски
uniq_propertyType_sum = df['propertyType'].nunique()
print(f'Количество уникальных значений столбца propertyType: {uniq_propertyType_sum}')

df['propertyType'].isnull().mean()*100

Количество уникальных значений столбца propertyType: 11


0.0

### Пропуски и обработка столбца street

In [243]:
df['street'].value_counts()
#print(df.loc[df['street'].isnull()])

street
Address Not Disclosed        672
Undisclosed Address          517
(undisclosed Address)        391
Address Not Available        175
Unknown Address               72
                            ... 
1737 Parkview Green Cir        1
14890 Rockridge Ln             1
497 Kingswood Dr               1
202 Seacrest Beach Blvd W      1
5983 Midcrown Dr               1
Name: count, Length: 337076, dtype: int64

Проанализировав данные можно увидеть, что основные значения это "адрес отсутствует" и т.п., имеющийся индекс не позволяет однозначно определить улицу. Дальнейшая обработка таких данных не представляется возможным и столбец следует удалить. 

In [244]:
df = df.dropna(subset=['street'])

### Пропуски и обработка столбца zipcode

In [245]:
df['zipcode'].value_counts()

zipcode
32137         2141
33131         1563
34747         1488
78245         1390
34759         1333
              ... 
27613-4237       1
43054            1
41630            1
27615-3709       1
64015            1
Name: count, Length: 4549, dtype: int64

В столбце нет незаполненных строк, это очень хорошо, но при дальнейшем исследование данных обнаруживаются не корректно внесенные данные, такие как объекты с двойным индексом или просто со значениями "--", "00000" или "0"<br>
Обработаем данные таким образом: удалим строки со значениями "--", "0" и "00000", а в строках с задвоенным индексом уберем второй индекс.

In [246]:
df = df.drop(df[(df['zipcode']=='--') | (df['zipcode']=='0')| (df['zipcode']=='00000')].index)

# уберем вторую часть индекса после дефиса
df['zipcode'] = df['zipcode'].str.replace("-.+",'', regex=True)

### Изучение и обработка столбца schools

In [247]:
# Проверим данные в столбце 
df['schools'].value_counts()

schools
[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                    4202
[{'rating': ['4/10', '5/10', '6/10'], 'data': {'Distance': ['39.69mi', '39.69mi', '39.69mi'], 'Grades': ['9-12', '6-8', 'PK-5']}, 'name': ['Fort Hancock High School', 'Fort Hancock Middle School', 'Benito Martinez Elementary School']}]                                                                                                                                                                                                                                        

Данные в этом столбце, которые как-либо могут повлиять на стоимость недвижимости, это рейтинг школы и расстояние до нее. Преобразуем информацию в новые признаки. 

In [248]:
# Отделим значение рейтинга
df['schools_rating'] = df['schools'].str.findall(r"\brating': ([\s\S]+?), 'data\b")

# Создадим функцию выделяющую числовые данные из вложенного списка
def choose_rating(data):
    data_n = data[0]
    return data_n

df['schools_rating'] = df['schools_rating'].apply(choose_rating)

# Создадим функцию что оставляет только числовые значения
def change_rating(rating):
    rating = rating.replace('[', '').replace(']', '').replace("'", '').replace('/10', '')
    return rating

df['schools_rating'] = df['schools_rating'].apply(change_rating)

In [249]:
# Определим среднее значение рейтинга
def find_average_rating(rating):
    rating = rating.split(', ')
    find_rating = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(find_rating) if find_rating else -1
    return average_rating

# Проиграем функцию, а полученные данные сохраним в отдельный столбец    
df['schools_rating'] = df['schools_rating'].apply(find_average_rating)

In [250]:
# Определим минимальную дистанцию до школы
distance_min = df['schools'].str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b") # выбираем расстояние
distance_min = distance_min.apply(lambda x: x[0]) # выбираем первый элемент из каждого списка
distance_min = distance_min.str.replace('[a-zA-Z]','', regex=True) # удаляем буквенные символы из каждой строки
distance_min = distance_min.str.findall(r'\b([0-9]+.[0-9]+)') # ищем все числа в каждой строке
distance_min = distance_min.apply(lambda x: [float(i) for i in x]) # преобразуем в float
school_distance_min = distance_min.apply(lambda x: -1 if len(x)==0 else min(x)) # находим мин. расстояние, если список пустой, то -1 

# Создадим признак school_distance_min
df['school_distance_min'] = school_distance_min

# Удаляем не нужные столбцы
df.drop(['schools', 'schools_rating'], axis=1, inplace=True)

### Изучение и обработка столбца homeFacts

In [251]:
# Проверим данные в столбце 
df['homeFacts'].value_counts()

homeFacts
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '—', 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                                       7174
{'atAGlanceFacts': [{'factValue': None, 'factLabel': 'Year built'}, {'factValue': None, 'factLabel': 'Remodeled year'}, {'factValue': None, 'factLabel': 'Heating'}, {'factValue': None, 'factLabel': 'Cooling'}, {'factValue': None, 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                            3532
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooli

Столбец сам по себе представляет значительное количество информации например такой как: дата постройки дома, наличие отопления, кондиционирования, наличие паркинга, площиди и даже цену за квадратный фут. Значение Цены за квадратный фут мы использовать не должны иначе образуется утечка данных при создании прогноза.
Изучим более подробно другие имеющиеся данные.

In [252]:
# Отделим значения Label и Value при помощи регулярных выражений и сведем их в новый список
Label = df['homeFacts'].str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
Value = df['homeFacts'].str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
label_list = ','.join(Label[0]).replace("'","").split(',')
label_list

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [253]:
# Добавим в датафрейм новые столбцы
for i, val in enumerate(label_list):
    df[val]=Value.apply(lambda x: x[i])

Анализ и обработка данных новом столбце Year built

In [254]:
# Проверим данные в столбце Year built
x = list(df['Year built'].unique())
print(x)

["'2019'", "'1961'", "'2006'", "''", "'1920'", "'1976'", "'1970'", "'1965'", "'2015'", "'1996'", "'1982'", "'1905'", "'2008'", "'1899'", "'2016'", "'1954'", "'1989'", "'2014'", "'1915'", "'1994'", "'1910'", "'1923'", "'1960'", "'1977'", "'1972'", "'1963'", "'1981'", "'1949'", "'1930'", "'1991'", "'1958'", "'2005'", "'2017'", "'2018'", "'1983'", "'1928'", "'1998'", "'1997'", "'1964'", "'1974'", "'1940'", "'1973'", "'1968'", "'1999'", "'1918'", "'2007'", "'1984'", "'1946'", "'2003'", "'2000'", "'2004'", "'1969'", "'2002'", "'1952'", "'1947'", "'1957'", "'1978'", "'1956'", "'1967'", "'1975'", "'1988'", "'1955'", "'1950'", "'1959'", "'2012'", "'1924'", "'1966'", "'1992'", "'2001'", "'1995'", "'1985'", "'2010'", "'1971'", "'1909'", "'1953'", "'1993'", "'1990'", "'1980'", "'1951'", "'1907'", "'1885'", "'1941'", "'1927'", "'1929'", "'1901'", 'None', "'1922'", "'1906'", "'1945'", "'1979'", "'2009'", "'1962'", "'1913'", "'2011'", "'1897'", "'1987'", "'2020'", "'1926'", "'1939'", "'1925'", "'201

In [255]:
# Удалим кавычки
df['Year built'] = df['Year built'].str.replace("'",'', regex=True)
x = list(df['Year built'].unique())
print(x)
df['Year built'].value_counts(sort=False)

['2019', '1961', '2006', '', '1920', '1976', '1970', '1965', '2015', '1996', '1982', '1905', '2008', '1899', '2016', '1954', '1989', '2014', '1915', '1994', '1910', '1923', '1960', '1977', '1972', '1963', '1981', '1949', '1930', '1991', '1958', '2005', '2017', '2018', '1983', '1928', '1998', '1997', '1964', '1974', '1940', '1973', '1968', '1999', '1918', '2007', '1984', '1946', '2003', '2000', '2004', '1969', '2002', '1952', '1947', '1957', '1978', '1956', '1967', '1975', '1988', '1955', '1950', '1959', '2012', '1924', '1966', '1992', '2001', '1995', '1985', '2010', '1971', '1909', '1953', '1993', '1990', '1980', '1951', '1907', '1885', '1941', '1927', '1929', '1901', 'None', '1922', '1906', '1945', '1979', '2009', '1962', '1913', '2011', '1897', '1987', '2020', '1926', '1939', '1925', '2013', '1986', 'No Data', '1943', '1935', '1890', '1900', '1895', '1911', '1948', '1942', '1916', '1931', '1944', '1893', '1879', '1914', '1937', '1912', '1938', '1921', '1860', '1850', '1936', '1861', 

Year built
2019    32168
1961     2054
2006     8008
        58725
1920     4329
        ...  
1019        1
1805        1
1843        1
1808        1
1804        1
Name: count, Length: 231, dtype: int64

In [256]:
# В столбце имеются некорректные данные изаменим их на 'unknown'
df['Year built'] = df['Year built'].str.replace('^\s*$','unknown', regex=True)
df['Year built'] = df['Year built'].str.replace('No Data','unknown')
df['Year built'] = df['Year built'].str.replace('559990649990','unknown')
df['Year built'] = df['Year built'].str.replace('^1$','unknown', regex=True)
df['Year built'] = df['Year built'].str.replace('None','unknown')

# Среди данных встречаются значения годов: '1060', '1019', '1057', '1208' вероятно это проявление человеческого фактора при заполнении данных и реальноимеется ввиду 1900 года и 2018 год, изменим их тоже
df['Year built'] = df['Year built'].str.replace('1060','1960')
df['Year built'] = df['Year built'].str.replace('1019','1919')
df['Year built'] = df['Year built'].str.replace('1057','1957')
df['Year built'] = df['Year built'].str.replace('1208','2018')

Анализ и обработка данных новом столбце  Remodeled year

In [257]:
# Проверим столбец Remodeled year.
x = list(df['Remodeled year'].unique()) 
print(x)

["''", "'1967'", "'2006'", 'None', "'2015'", "'1997'", "'2019'", "'1982'", "'1905'", "'2009'", "'2007'", "'1996'", "'1995'", "'1988'", "'1972'", "'1949'", "'1954'", "'1930'", "'2003'", "'2000'", "'1994'", "'2005'", "'1983'", "'1965'", "'1968'", "'1993'", "'1999'", "'2014'", "'2008'", "'2010'", "'2004'", "'2002'", "'1984'", "'1952'", "'1978'", "'1975'", "'1971'", "'1947'", "'1957'", "'1950'", "'2012'", "'1973'", "'1970'", "'2016'", "'1989'", "'1998'", "'1980'", "'1986'", "'1955'", "'1991'", "'2011'", "'2017'", "'1941'", "'2001'", "'1990'", "'1979'", "'1901'", "'1918'", "'1977'", "'1962'", "'1956'", "'2013'", "'1960'", "'1981'", "'1969'", "'1987'", "'1958'", "'1910'", "'1925'", "'1964'", "'1966'", "'1992'", "'1943'", "'1961'", "'1948'", "'1931'", "'1976'", "'1974'", "'1935'", "'1940'", "'2018'", "'1916'", "'1914'", "'1963'", "'1985'", "'1951'", "'1944'", "'1942'", "'1959'", "'1927'", "'1946'", "'1945'", "'1920'", "'1938'", "'1936'", "'0'", "'1939'", "'1915'", "'1928'", "'1923'", "'1929'"

In [258]:
# Удалим кавычки и некорректные значения
df['Remodeled year'] = df['Remodeled year'].str.replace("'",'', regex=True)
df['Remodeled year'] = df['Remodeled year'].str.replace('^0$', 'None', regex=True)
df['Remodeled year'] = df['Remodeled year'].str.replace('1111', 'None', regex=True)
df['Remodeled year'] = df['Remodeled year'].str.replace('^\s*$','None', regex=True)


In [259]:
# Нас интересует былал ли вообще на объекте реконструкция из-за этого данные будет проще превратить в бинарный вид
def remodelbool(value):
    if isinstance(value, str) and re.match(r'^\d{4}$', value):
        return True
    else:
        return False
# применяем функцию
df['Remodeled year_final'] = df['Remodeled year'].apply(remodelbool)

Анализ и обработка нового столбца Heating

In [260]:
# Проверим столбец
df['Heating'].str.lower().value_counts()

Heating
'forced air'                                                      134307
''                                                                105753
'other'                                                            29622
'electric'                                                         10216
'gas'                                                               9296
                                                                   ...  
'zoned heating, wall unit heating, forced air heating'                 1
'baseboard, spacewallunit'                                             1
'hot air, stove-pellet'                                                1
'natural gas, space heater'                                            1
'baseboard, hot water, programmable thermostat, radiant floor'         1
Name: count, Length: 1919, dtype: int64

In [261]:
# Удалим кавычки
df['Heating'] = df['Heating'].str.replace("'",'', regex=True)

Данные в столбце содержат информацию о видах отопления. Для наших задачь будет достаточно информации в виде бинарных значений True - отопление есть, False - отопления нет.

In [264]:
# Изменяем данные на True и False
df['Heating_final'] = df['Heating'].apply(lambda x: True if x not in ['', 'No Data', 'No Heat Fuel', 'No Heat', 'None'] else False)
df['Heating_final'].value_counts()

Heating_final
True     259154
False    117973
Name: count, dtype: int64

Анализ и обработка нового столбца Cooling

In [266]:
# посмотрим содержание столбца
df['Cooling'].str.lower().value_counts()

Cooling
'central'                                                                158743
''                                                                       120390
'central air'                                                             14384
'no data'                                                                 10615
'has cooling'                                                              9730
                                                                          ...  
'central gas, propane, zoned'                                                 1
'other (see remarks), panel/floor/wall, window unit'                          1
'multi units, zoned cooling'                                                  1
'central air, g-energy star hvac, gas hot air/furnace, multizone a/c'         1
'central a/c (gas), central heat (gas), heat pump'                            1
Name: count, Length: 1439, dtype: int64

В данных этого столбца находятся слишком не нормализованные данные и о наличии кондиционирования и об отопление. <br>
Проверим количество не заполненных данных.<br>

In [271]:
# Проверим процентное соотношение незаполненных данных в столбце
cooling_nodata = round(((df[df['Cooling']==''].shape[0]) / (df['Cooling'].shape[0]) * 100), 1)
print(f'Пропущенные данные в столбце Cooling: {cooling_nodata} %')

Пропущенные данные в столбце Cooling: 31.9 %


Нормализация данных в этом столбце займет большое количество времени, а данные изв нем не значительны. Логично его удалить.<br>

In [272]:
# Удаляем не нужные столбцы
df.drop(['Cooling'], axis=1, inplace=True)

Анализ и обработка данных в столбце Parking

In [274]:
# Проверим данные в столбце
df['Parking'].str.lower().value_counts()

Parking
''                                                                                                                                    171839
'attached garage'                                                                                                                      70748
'2 spaces'                                                                                                                             28061
'1 space'                                                                                                                              14252
'no data'                                                                                                                              13332
                                                                                                                                       ...  
'carport - 2 car, detached garage, garage - 1 car, side load garage'                                                                       1
'atta

In [277]:
# уберем лишние кавычки и изучим подробнее содержание
df['Parking'] = df['Parking'].str.replace("'",'', regex=True)
print(list(df['Parking'].unique()))

['', 'Attached Garage', 'Detached Garage', 'Carport, Attached Garage', '2 spaces', '4 spaces', 'Off Street', 'None', '0', '1 space', 'No Data', 'Carport', '2', '3 spaces', 'Double Wide Drive, Oversized Drive', 'Attached Garage, Detached Garage, Carport', '1', 'Covered Lot', '10', '7', 'Detached Garage, Attached Garage', 'On Street, Detached Garage, Carport', 'Attached-Detached, ASPHT, GRAVL', 'Garage Type', 'Curb Parking, Garage Door Opener', 'Detached parking', '7 spaces', 'Slab Parking Spaces', 'Parking YN', '"Attchd Gar Cabinets, Electric Door Opener, Dir Entry frm Garage, Carport Spaces"', 'On street, Attached Garage', 'Asphalt Driveway, Off Street', 'Electric Door Opener, Extnded Lngth Garage, Tandem Garage, Dir Entry frm Garage, Slab Parking Spaces', '2 Car Garage, Attached, Main Level, Off Street Parking', 'Off street', 'Garage - 2 Car', 'Attached Garage, Driveway', 'Parking Desc', 'Off street, On street, Attached Garage', '4', 'Electric Door Opener, Over Height Garage, RV Gate,

In [282]:
# Проверим процентное соотношение незаполненных данных в столбце
Parking_nodata = round(((df[df['Parking']==''].shape[0]) / (df['Parking'].shape[0]) * 100), 1)
print(f"Процент не заполненных строк в столбце Parking: {Parking_nodata} %")

Процент не заполненных строк в столбце Parking: 45.6 %


Процент пропущенных данных приближается к 50 %, а еще есть значительная часть данных с некорректными значениями...<br>
Столбец следует удалить так как он не несет значительнйо смысловой нагрузки из-за отсутствия большого количества данных, а так же не соответствияе данных заявленному названию столбца.

In [ ]:
# Удаляем не нужные столбцы
df.drop(['Parking'], axis=1, inplace=True)

Столбец lotsize показывает площадь дома или участка, для нашей задачи информация не интересная, удаляем его.<br>
Столбец Price/sqft создает утечку данных, удаляем его.

In [283]:
# удаляем лишние и отработанные столбцы
df = df.drop(['homeFacts', 'Remodeled year', 'lotsize', 'Price/sqft', 'Heating'], axis=1)

### Обработка целевого признака target

In [284]:
# Проверим содержание таргета
df['target'].unique()[0:10]

array(['$418,000', '$310,000', '$2,895,000', '$2,395,000', '$5,000',
       '$209,000', '181,500', '68,000', '$244,900', '$311,995'],
      dtype=object)

In [286]:
# Проверим процентное соотношение пропусков в тарегете
round((df['target'].isnull().mean()*100), 2)

0.66


В столбце таргете имеются, в добавок строки, где нет таргета не информативны в модели регрессии, так как у нас нет возможности оценить качество модели. Поэтому эти строки удаляем.

In [287]:
# Удаляем строки, где нет таргета
df = df.dropna(subset=['target'])

In [288]:
# Уберем излишние символы
df['target'] = df['target'].replace('\$', '', regex=True)
df['target'] = df['target'].replace(',', '', regex=True)
df['target'] = df['target'].replace('\+', '', regex=True)

В данных имеется значение "/mo", предположим чот это цена за месяц. Проверим статус таких объектов. Цена указанная в таком формате для наших дальнейших целей не подойдет - удаляем строки с такими данными<br>

In [289]:
# Статус объектов с ценой /mo
df[df['target'].str.contains('/mo',regex=True)].head()

# статус таких квартир For rent
# удаляем эти строки
df = df[~df['target'].str.contains('/mo', regex=True)]

In [57]:
# Переведем столбец в числовое значение
df['target'] = df['target'].astype(int)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374249 entries, 0 to 377134
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                374249 non-null  object 
 1   propertyType          374249 non-null  object 
 2   street                374249 non-null  object 
 3   baths                 374249 non-null  float64
 4   city                  374249 non-null  object 
 5   sqft                  374249 non-null  int32  
 6   zipcode               374249 non-null  object 
 7   state                 374249 non-null  object 
 8   target                374249 non-null  int32  
 9   private_pool_final    374249 non-null  bool   
 10  beds_final            374249 non-null  float64
 11  schools_rating        374249 non-null  float64
 12  school_distance_min   374249 non-null  float64
 13  Year built            374249 non-null  object 
 14  Remodeled year_final  374249 non-null  bool   
 15  

## *Выводы по первой части работы:*
- Подробно изучили и проанализированли предоставленные данные,
- Преобразовали и по возможности дополнили данные,
- Сократили количество полезных столбцов, 
- Почистили данные от дубликатов,
- Избавились от избыточных данных
- Данные сохраняем в файл, чтоб далее было удобнее работать

In [292]:
# сохраняем предобработанные данные в CSV-файл для упрощения дальнейшей работы
df.to_csv(r"D:\Diplom\cleaned_data.csv", index=False)